# Week 4 - Models and Experimentation

## Step 1 Training a model

For the purposes of this demo, we will be using this [adapted demo](https://www.datacamp.com/tutorial/xgboost-in-python) and training an XGBoost model, and then doing some experimentation and hyperparameter tuning.


If running this notebook locally, use the following steps to create virtual environment:
- Don't use past python 3.10
- To create virtual environment use "venv"

`python -m venv NAME`

- Try to avoid anaconda, poetry or similar package management platforms
- To install a package use pip

`python -m pip install <package-name>`

- once you are done working with this virtual environment, deactivate it with `deactivate`

### Install packages

In [1]:
# !pip install wandb -qU

In [2]:
import xgboost as xgb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


### Import data

We will be using Diamonds dataset imported from Seaborn. It is also available on [Kaggle](https://www.kaggle.com/datasets/shivam2503/diamonds).

Read about the features by following the link. We will be predicting the price of diamonds.

In [3]:
diamonds = sns.load_dataset('diamonds')
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   carat    53940 non-null  float64 
 1   cut      53940 non-null  category
 2   color    53940 non-null  category
 3   clarity  53940 non-null  category
 4   depth    53940 non-null  float64 
 5   table    53940 non-null  float64 
 6   price    53940 non-null  int64   
 7   x        53940 non-null  float64 
 8   y        53940 non-null  float64 
 9   z        53940 non-null  float64 
dtypes: category(3), float64(6), int64(1)
memory usage: 3.0 MB


In [5]:
diamonds.shape

(53940, 10)

In [6]:
X,y = diamonds.drop('price', axis=1), diamonds[['price']]

# For the cut, color and clarity use pandas category to enable XGBoost ability to deal with categorical data.

X['cut'] = X['cut'].astype('category')
X['color'] = X['color'].astype('category')
X['clarity'] = X['clarity'].astype('category')

### Split the data and train a model

In [7]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(X_test, label=y_test, enable_categorical=True)

In [8]:
# Define hyperparameters
# Changing tree method
params = {"objective": "reg:squarederror", "tree_method": "hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
)

In [9]:
# Define evaluation metrics - Root Mean Squared Error

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

RMSE: 545.191877397669


### Incorporate validation

In [10]:
params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 100

# Create the validation set
evals = [(dtrain, "train"), (dtest, "validation")]

In [11]:
evals = [(dtrain, "train"), (dtest, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
)

[0]	train-rmse:2861.71326	validation-rmse:2853.85688
[10]	train-rmse:554.29819	validation-rmse:579.26422
[20]	train-rmse:493.68077	validation-rmse:547.75493
[30]	train-rmse:467.32713	validation-rmse:540.03567
[40]	train-rmse:447.40974	validation-rmse:541.70531
[50]	train-rmse:432.62075	validation-rmse:540.89769
[60]	train-rmse:422.28318	validation-rmse:540.63039
[70]	train-rmse:410.72350	validation-rmse:543.67077
[80]	train-rmse:398.24619	validation-rmse:545.08296
[90]	train-rmse:386.92486	validation-rmse:543.90036
[99]	train-rmse:379.58717	validation-rmse:545.19188


In [12]:
# Incorporate early stopping
n = 10000


model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50
)

[0]	train-rmse:2861.71326	validation-rmse:2853.85688


[50]	train-rmse:432.62075	validation-rmse:540.89769
[83]	train-rmse:393.82435	validation-rmse:544.68591


In [13]:
# Cross-validation

params = {"objective": "reg:squarederror", "tree_method": "hist"}
n = 1000

results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)


In [14]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2861.512810,8.494816,2861.704341,37.144992
1,2081.847733,5.811005,2084.838207,31.889208
2,1547.031906,5.092391,1554.657450,30.699908
3,1184.129738,3.982239,1194.251600,26.940062
4,942.998782,3.327174,960.239319,24.392689


In [15]:
best_rmse = results['test-rmse-mean'].min()

best_rmse

553.4613038243663

## Start W&B


- Login into your W&B profile using the code below
- Alternatively you can set environment variables. There are several env variables which you can set to change the behavior of W&B logging. The most important are:
    - WANDB_API_KEY - find this in your "Settings" section under your profile
    - WANDB_BASE_URL - this is the url of the W&B server

- Find your API Token in "Profile" -> "Setttings" in the W&B App



In [16]:
# Log in to your W&B account
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mudit14224 (muditjindal2025). Use `wandb login --relogin` to force relogin


True

In [17]:
# TO DO
# Start experiment tracking with W&B
# Do at least 5 experiments with various hyperparameters
# Choose any method for hyperparameter tuning: grid search, random search, bayesian search
# Describe your findings and what you see

In [17]:
from wandb.xgboost import WandbCallback

In [24]:
wandb.init(
    project="xgboost-runs",
    name="run_1",
    config={
        "model": "xgboost",
        "dataset": "diamonds",
    }
)
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "hist"}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   callbacks=[WandbCallback(log_model=True)]
)

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")
wandb.log({"RMSE": rmse})

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
epoch,49


RMSE: 545.191877397669


In [28]:
# Changing the hyperparameters
wandb.init(
    project="xgboost-runs",
    name="run_2",
    config={
        "model": "xgboost",
        "dataset": "diamonds",
        "max_depth": 5,
        "alpha": 10,
        "learning_rate": 0.1
    }
)
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "hist", 
          "max_depth":5, "alpha":10, "learning_rate": 0.1}

n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   callbacks=[WandbCallback(log_model=True)]
)

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")
wandb.log({"RMSE": rmse})


RMSE Test,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train-rmse,████▇▇▇▇▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
validation-rmse,████▇▇▇▇▆▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁
RMSE Test,3029.40748
epoch,49


RMSE: 535.4199312398769


In [29]:
# With Validation
# Changing the hyperparameters
wandb.init(
    project="xgboost-runs",
    name="run_3",
    config={
        "model": "xgboost",
        "dataset": "diamonds",
        "max_depth": 10,
        "alpha":2,
        "learning_rate": 0.01,
        "colsample_bytree": 0.3
    }
)
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "hist", 
          "max_depth":10, "alpha":2, "learning_rate": 0.01, "colsample_bytree":0.3}


model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10,
   callbacks=[WandbCallback(log_model=True)]
)

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE Test: {rmse}")
wandb.log({"RMSE Test": rmse})

RMSE,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
RMSE,535.41993
epoch,99


[0]	train-rmse:3954.93873	validation-rmse:3951.99883
[10]	train-rmse:3712.96115	validation-rmse:3709.14646
[20]	train-rmse:3518.23981	validation-rmse:3514.00305
[30]	train-rmse:3309.39194	validation-rmse:3305.18958
[40]	train-rmse:3166.87327	validation-rmse:3163.34468
[50]	train-rmse:3009.01259	validation-rmse:3005.61437
[60]	train-rmse:2840.77077	validation-rmse:2837.54104
[70]	train-rmse:2666.08798	validation-rmse:2663.38948
[80]	train-rmse:2509.14655	validation-rmse:2506.88058
[90]	train-rmse:2400.31383	validation-rmse:2399.13194
[99]	train-rmse:2262.93799	validation-rmse:2263.02354
RMSE Test: 2263.023545825612


In [38]:
# With Validation and Early stopping
# Changing the hyperparameters 
wandb.init(
    project="xgboost-runs",
    name="run_4",
    config={
        "model": "xgboost",
        "dataset": "diamonds",
        "max_depth": 5,
        "alpha": 10,
        "learning_rate": 0.1
    }
)
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "hist", 
          "max_depth":5, "alpha":10, "learning_rate": 0.1}

n = 10000
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50, 
   callbacks=[WandbCallback(log_model=True)]
)

n = 1000
results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20, 
)

print(results.head())

best_rmse = results['test-rmse-mean'].min()
print(f"Best RMSE: {best_rmse}")
wandb.log({"Best RMSE": best_rmse})

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")
wandb.log({"RMSE": rmse})


best_iteration,▁
best_score,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train-rmse,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation-rmse,█▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_iteration,296
best_score,528.89675
epoch,346


[0]	train-rmse:3617.36207	validation-rmse:3614.31160
[50]	train-rmse:532.98165	validation-rmse:558.32549
[100]	train-rmse:488.31206	validation-rmse:535.54305
[150]	train-rmse:469.11783	validation-rmse:531.56285
[200]	train-rmse:452.95420	validation-rmse:531.31608
[250]	train-rmse:442.64181	validation-rmse:529.85469
[300]	train-rmse:429.65673	validation-rmse:529.68941
[346]	train-rmse:420.03540	validation-rmse:530.73783
   train-rmse-mean  train-rmse-std  test-rmse-mean  test-rmse-std
0      3617.399812        9.942667     3617.500415      39.579597
1      3283.117437        8.650593     3282.921723      37.844734
2      2984.170973        8.103091     2984.550998      35.925090
3      2715.492138        7.515823     2715.649351      34.273621
4      2474.615568        6.196544     2475.692257      33.734063
Best RMSE: 539.1346308606683
RMSE: 530.7378283475853


In [40]:
# With Validation and Early stopping
# Changing the hyperparameters 
wandb.init(
    project="xgboost-runs",
    name="run_5",
    config={
        "model": "xgboost",
        "dataset": "diamonds",
        "max_depth": 15,
        "alpha": 15,
        "learning_rate": 0.01,
    }
)
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "hist", 
          "max_depth":15, "alpha":15, "learning_rate": 0.01}

n = 10000
model = xgb.train(
   params=params,
   dtrain=dtrain,
   num_boost_round=n,
   evals=evals,
   verbose_eval=50,
   # Activate early stopping
   early_stopping_rounds=50, 
   callbacks=[WandbCallback(log_model=True)]
)

n = 1000
results = xgb.cv(
   params, dtrain,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20, 
)

print(results.head())

best_rmse = results['test-rmse-mean'].min()
print(f"Best RMSE: {best_rmse}")
wandb.log({"Best RMSE": best_rmse})

predictions = model.predict(dtest)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")
wandb.log({"RMSE": rmse})

best_iteration,▁
best_score,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train-rmse,█▇▇▆▅▄▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation-rmse,█▇▆▅▅▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_iteration,535
best_score,555.67805
epoch,584


[0]	train-rmse:3951.25048	validation-rmse:3948.54234
[50]	train-rmse:2442.17557	validation-rmse:2445.41026
[100]	train-rmse:1536.80062	validation-rmse:1558.23406
[150]	train-rmse:993.83397	validation-rmse:1049.57038
[200]	train-rmse:666.55600	validation-rmse:779.40709
[250]	train-rmse:466.72463	validation-rmse:647.26745
[300]	train-rmse:340.13265	validation-rmse:589.81566
[350]	train-rmse:261.19557	validation-rmse:566.59121
[400]	train-rmse:209.79646	validation-rmse:558.78468
[450]	train-rmse:175.52816	validation-rmse:556.54407
[500]	train-rmse:150.34190	validation-rmse:556.06570
[550]	train-rmse:131.38757	validation-rmse:555.80062
[584]	train-rmse:121.68792	validation-rmse:556.15525
   train-rmse-mean  train-rmse-std  test-rmse-mean  test-rmse-std
0      3951.281557       10.503417     3951.281702      42.367674
1      3913.024225       10.397055     3913.149854      42.169966
2      3875.153082       10.291542     3875.404246      41.978591
3      3837.668613       10.188698     3838

### Using Sweep for hyperparamter tuning

In [18]:
sweep_config = {
    "method": "grid", 
    "metric": {
      "name": "rmse",
      "goal": "minimize"   
    },
    "parameters": {
        "booster": {
            "values": ["gbtree"]
        },
        "max_depth": {
            "values": [3, 6, 9, 12]
        },
        "learning_rate": {
            "values": [0.01, 0.05, 0.1]
        },
        "alpha": {
            "values": [5, 10, 15]
        }
    }
}

In [19]:
sweep_id = wandb.sweep(sweep_config, project="xgboost-runs")

Create sweep with ID: gl79h1vt
Sweep URL: https://wandb.ai/muditjindal2025/xgboost-runs/sweeps/gl79h1vt


In [20]:
def train():
    wandb.init(
    project="xgboost-runs",
    name="sweep_grid",
    config={
        "model": "xgboost",
        "dataset": "diamonds",
        "max_depth": 15,
        "alpha": 15,
        "learning_rate": 0.01,
        "booster": "gbtree"
        },
    )

    config = wandb.config

    # Define hyperparameters
    params = {"objective": "reg:squarederror", "tree_method": "hist", 
            "max_depth":config.max_depth, "alpha":config.alpha, "learning_rate": config.learning_rate}
    
    n = 1000
    model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=n,
    evals=evals,
    verbose_eval=100,
    callbacks=[WandbCallback(log_model=True)]
    )

    predictions = model.predict(dtest)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    print(f"RMSE Test: {rmse}")
    

In [21]:
wandb.agent(sweep_id, train, count=25)

wandb: Agent Starting Run: nwqtrzqx with config:
wandb: 	alpha: 5
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 3
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[0]	train-rmse:3954.54101	validation-rmse:3951.56348
[100]	train-rmse:1832.69388	validation-rmse:1819.84126
[200]	train-rmse:1136.34498	validation-rmse:1122.50574
[300]	train-rmse:904.31723	validation-rmse:890.55875
[400]	train-rmse:794.16010	validation-rmse:781.74851
[500]	train-rmse:721.92676	validation-rmse:713.63562
[600]	train-rmse:678.71685	validation-rmse:673.17272
[700]	train-rmse:650.90001	validation-rmse:647.86982
[800]	train-rmse:631.59914	validation-rmse:631.25784
[900]	train-rmse:618.07655	validation-rmse:620.58826
[999]	train-rmse:607.54228	validation-rmse:612.07293
RMSE Test: 612.0729261853002


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train-rmse,█▆▅▄▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation-rmse,█▆▅▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,999


wandb: Agent Starting Run: gabzg7jy with config:
wandb: 	alpha: 5
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 6
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[0]	train-rmse:3951.98716	validation-rmse:3949.09101
[100]	train-rmse:1604.09045	validation-rmse:1599.23608
[200]	train-rmse:812.22789	validation-rmse:814.55972
[300]	train-rmse:583.93393	validation-rmse:602.22037
[400]	train-rmse:522.19358	validation-rmse:552.36202
[500]	train-rmse:499.01695	validation-rmse:537.55716
[600]	train-rmse:485.68104	validation-rmse:532.11116
[700]	train-rmse:473.61262	validation-rmse:528.89390
[800]	train-rmse:464.24335	validation-rmse:527.04793
[900]	train-rmse:456.55593	validation-rmse:525.59245
[999]	train-rmse:450.80299	validation-rmse:524.80236
RMSE Test: 524.8023629451269


...

[0]	train-rmse:3603.47145	validation-rmse:3600.98051
[100]	train-rmse:163.05486	validation-rmse:549.25543
[200]	train-rmse:103.46439	validation-rmse:553.52769
[300]	train-rmse:71.45506	validation-rmse:555.45751
[400]	train-rmse:50.59752	validation-rmse:556.47542
[500]	train-rmse:37.10462	validation-rmse:557.11638
[600]	train-rmse:28.47530	validation-rmse:557.70723
[700]	train-rmse:21.93388	validation-rmse:558.00798
[800]	train-rmse:17.86062	validation-rmse:558.20313
[900]	train-rmse:15.33451	validation-rmse:558.32738
[999]	train-rmse:14.08646	validation-rmse:558.40510
RMSE Test: 558.4051036618046


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train-rmse,█▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation-rmse,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,999


wandb: Agent Starting Run: ajd806k7 with config:
wandb: 	alpha: 15
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.01
wandb: 	max_depth: 3
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[0]	train-rmse:3954.54101	validation-rmse:3951.56348
[100]	train-rmse:1832.69998	validation-rmse:1819.86908
[200]	train-rmse:1136.86561	validation-rmse:1122.88619
[300]	train-rmse:904.80942	validation-rmse:891.45887
[400]	train-rmse:793.88297	validation-rmse:782.34410
[500]	train-rmse:722.53339	validation-rmse:714.78765
[600]	train-rmse:679.54389	validation-rmse:674.77661
[700]	train-rmse:651.09714	validation-rmse:648.80248
[800]	train-rmse:631.65158	validation-rmse:631.88720
[900]	train-rmse:618.16837	validation-rmse:621.00523
[999]	train-rmse:607.45041	validation-rmse:612.26175
RMSE Test: 612.2617532714547


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train-rmse,█▆▅▄▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation-rmse,█▆▅▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,999


## XGBoost Hyperparameter Tuning Experiment

In this experiment, we utilized the XGBoost machine learning framework to optimize hyperparameters aimed at minimizing the root mean square error (RMSE) on a validation dataset. Our hyperparameter tuning approach was conducted through a grid search method, evaluating various combinations of parameters.

### Configuration Tested

- **Booster**: Utilized gbtree (gradient boosted tree), which builds tree-based models for predictions.
- **Max Depth**: Tested at four levels (3, 6, 9, 12) to determine how the depth of the trees affects model performance.
- **Learning Rate**: Evaluated at three levels (0.01, 0.05, 0.1) to find the optimal step size shrinkage used to prevent overfitting.
- **Alpha**: This L1 regularization term on weights was assessed at three levels (5, 10, 15) to help control model complexity.

### Optimal Hyperparameters

After conducting the grid search across all combinations of these parameters, the optimal set that emerged was:

- **Alpha**: 10
- **Learning Rate**: 0.1
- **Max Depth**: 6

This configuration yielded a validation RMSE of 524.121, indicating the most effective combination for minimizing prediction errors on our validation dataset. This optimal result will guide the model's deployment strategy and further testing phases.
